In [2]:
# !pip install transformers
# !pip install accelerate

In [3]:
import numpy as np
import pandas as pd
import torch
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification
from torch.utils.data import Dataset, DataLoader

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

In [ ]:
reviews = pd.read_csv("/content/drive/MyDrive/Transformers/Sentiment_Analysis/Movies_Dataset/movie_data.csv")
reviews.head()

In [ ]:
test_text = reviews.iloc[40000:, 0].tolist()
test_label = reviews.iloc[40000:, 1].values

In [12]:
model_name = "siebert/sentiment-roberta-large-english"

In [ ]:
tokenizer = RobertaTokenizerFast.from_pretrained(model_name)

In [ ]:
test_embedding = tokenizer(test_text, padding = True, truncation = True, max_length = 512)

In [ ]:
class ReviewsDataset(Dataset):
    def __init__(self, embedding, label):
        self.embedding = embedding
        self.label = label

    def __len__(self):
        return len(self.label)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.embedding.items()}
        item["label"] = torch.tensor(self.label[idx])
        return item

In [ ]:
test_dataset = ReviewsDataset(test_embedding, test_label)

In [ ]:
test_loader = DataLoader(test_dataset, batch_size = 32, shuffle = True, drop_last = True)

In [ ]:
model = RobertaForSequenceClassification.from_pretrained(model_name)

In [ ]:
model.to(device)

In [ ]:
def check_accuracy(model, data_loader, device):

    with torch.no_grad():

        correct, total = 0, 0

        for batch in data_loader:

            embeddings = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)

            outputs = model(embeddings, attention_mask = attention_mask, labels = labels)
            loss, logits = outputs["loss"], outputs["logits"]

            _, predicted_labels = torch.max(logits, 1)
            correct += (predicted_labels == labels).sum()
            total += labels.size(0)

    return (correct.float() / total) * 100

In [ ]:
model.eval()
model.to(device)

print(f'Test accuracy: {check_accuracy(model, test_loader, device):.2f}%')